In [1]:
from pathlib import Path

In [13]:
tv_dir_path =Path(r'D:\data\dpe_full\dpe_doc\tv_tables')
tr_dir_path =Path(r'D:\data\dpe_full\dpe_doc\tr_tables')

In [14]:
trtv_dir_path=tv_dir_path

In [15]:
    trtv_dir_path = Path(trtv_dir_path)
    trtv_table_dict = dict()
    for table_path in trtv_dir_path.iterdir():
        table_name = table_path.name.split('.csv')[0]
        table = pd.read_csv(table_path).astype('category')
        ids = [col for col in table if col.endswith('id')]
        table[ids] = table[ids].astype(str).astype('category')
        table_ref = table_path.name.split('_')[0]
        table.columns = [table_ref + '_' + el if not el.startswith(('tr', 'tv')) else el for el in table.columns]
        table = table.rename(columns={f'{table_ref}_id': f'{table_name}_id'})
        trtv_table_dict[table_name] = table

In [12]:
trtv_dir_path = Path(trtv_dir_path)


tv025_type_batiment not found
tv025_type_emetteur not found
tv025_equipement_intermittence not found
tv029_simu_type_distribution not found


In [15]:
def load_tvtr_table_dict(tvtr_dir_path):

    tvtr_dir_path = Path(tvtr_dir_path)
    tvtr_table_dict =dict()
    for table_path in tvtr_dir_path.iterdir():
        table_name =table_path.name.split('.csv')[0]
        table = pd.read_csv(table_path).astype('category')
        table_ref = table_path.name.split('_')[0]
        table.columns = [table_ref+'_'+el if not el.startswith(('tr','tvtr')) else el for el in table.columns ]
        table = table.rename(columns ={f'{table_ref}_id':f'{table_name}_id'})
        tvtr_table_dict[table_name] = table 
    for table_name,table in tvtr_table_dict.items():

        foreign_ids = [col for col in table if col.endswith('_id') and not col.startswith(table_name)]
        for foreign_id in foreign_ids:
            foreign_table_name = foreign_id.split('_id')[0]
            foreign_table=tvtr_table_dict.get(foreign_table_name,None)
            if foreign_table is None:
                print(foreign_table_name+' not found')
            else:
                table = table.merge(foreign_table,on = foreign_id,how='left')
        cols = [el for el in table if not any(s in el for s in ['simulateur','simu_ordre','id'])]
        cols.append(f'{table_name}_id')
        tvtr_table_dict[table_name] =table[cols]
    return tvtr_table_dict

In [26]:

class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]

class DPEMetaData(metaclass=Singleton):

    def __init__(self ,assets_dir=Path(r'D:\data\dpe_full\dpe_doc')):

        self.tv_table_dict = load_tv_table_dict(assets_dir/'tv_tables')
        self.tr_table_dict = load_tr_table_dict(assets_dir/'tr_tables')


    def merge_all_tr_table(self, table):
        table = table.copy()
        tr_cols = [col for col in table.columns if col.startswith('tr')]
        for tr_col in tr_cols:
            table = self.merge_tr_table(table, tr_col)
        return table



    def merge_all_tv_table(self, table):
        table = table.copy()
        tv_cols = [col for col in table.columns if col.startswith('tv')]
        for tv_col in tv_cols:
            table = self.merge_tv_table(table, tv_col)
        return table
    
    def merge_tv_table(self, table, tv_col):
        table.merge(self.tv_table_dict[])
        tv_table_name = tv_col.split('_id')[0]
        tv_table = self.tv_table_dict[tv_table_name]
        table = table.merge(tv_table, on=tv_col, how='left')
        table.index = table.index.astype(stv)
        cols = tv_table.columns.tolist()
        table[cols]=table[cols].astype('category')
        return table




In [23]:
td007

,id,td006_batiment_id,tr014_type_parois_opaque_id,reference,deperdition_thermique,tv001_coefficient_reduction_deperditions_id,tv002_local_non_chauffe_id,coefficient_transmission_thermique_paroi,coefficient_transmission_thermique_paroi_non_isolee,tv003_umur_id,tv004_umur0_id,tv005_upb_id,tv006_upb0_id,tv007_uph_id,tv008_uph0_id,resistance_thermique_isolation,epaisseur_isolation,surface_paroi
0,1048,209,1,Mur,40.27,1,NaN,0.35,0.00,NaN,78,NaN,NaN,NaN,NaN,0.00,10.00,115.06
1,1049,209,1,Mur,3.59,NaN,1,0.35,0.00,NaN,78,NaN,NaN,NaN,NaN,0.00,10.00,10.80
2,1050,209,4,Plafond,6.49,NaN,5,0.14,0.00,NaN,NaN,NaN,NaN,NaN,10,0.00,27.00,49.10
3,1814,364,1,Mur,31.71,1,NaN,0.38,0.00,1,78,NaN,NaN,NaN,NaN,0.00,9.00,82.67
4,1815,364,1,Mur,3.89,NaN,1,0.32,0.00,1,1,NaN,NaN,NaN,NaN,0.00,0.00,12.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
439933,24115581,5419822,3,Plancher,17.35,261,NaN,0.25,0.00,NaN,NaN,85,9,NaN,NaN,0.00,0.00,69.40
439934,24115582,5419822,1,Mur,45.94,1,NaN,0.40,0.00,51,78,NaN,NaN,NaN,NaN,0.00,10.00,114.86
439935,24115588,5419822,1,Mur,5.60,NaN,1,0.40,0.00,51,69,NaN,NaN,NaN,NaN,0.00,0.00,17.50
439936,24115589,5419822,4,Plafond,9.33,1,NaN,0.23,0.00,NaN,NaN,NaN,NaN,99,10,0.00,0.00,40.55


In [1]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

In [2]:
from assets_orm import DPEMetaData
meta = DPEMetaData()

tv013_valeur_pont_thermique_isolation_planche_bas not found
tv025_type_batiment not found
tv025_type_emetteur not found
tv025_equipement_intermittence not found
tv026_classe_inertie_plancher_bas not found
tv026_classe_inertie_plancher_haut not found
tv026_classe_inertie_paroi_verticale not found
tv026_classe_inertie_classe_inertie not found
tv029_simu_type_distribution not found


In [195]:
td011 =pd.read_csv(data_dir/'depts'/dept/'td011_installation_chauffage.csv',dtype=str)
td012 =pd.read_csv(data_dir/'depts'/dept/'td012_generateur_chauffage.csv',dtype=str)
td013 =pd.read_csv(data_dir/'depts'/dept/'td013_installation_ecs.csv',dtype=str)
td014 =pd.read_csv(data_dir/'depts'/dept/'td014_generateur_ecs.csv',dtype=str)
td006 =pd.read_csv(data_dir/'depts'/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/'depts'/dept/'td001_dpe.csv',dtype=str)


In [18]:
td008.dtypes.astype(str).to_dict()

{'id': 'object',
 'td007_paroi_opaque_id': 'object',
 'reference': 'object',
 'td008_baie_id': 'object',
 'deperdition': 'object',
 'tv009_coefficient_transmission_thermique_vitrage_id': 'object',
 'presence_survitrage': 'object',
 'coefficient_transmission_thermique_baie': 'object',
 'tv010_coefficient_transmission_thermique_baie_id': 'object',
 'tv011_resistance_additionnelle_id': 'object',
 'tv012_coef_transmission_thermique_baie_protection_solaire_id': 'object',
 'surface': 'object',
 'perimetre': 'object',
 'tv013_valeur_pont_thermique_id': 'object',
 'facteur_solaire': 'object',
 'tv021_facteur_solaire_id': 'object',
 'tv022_coefficient_masques_proches_id': 'object',
 'coefficient_masques_lointains_non_homogenes': 'object',
 'tv023_coefficient_masques_lointains_homogenes_id': 'object',
 'tv020_coefficient_orientation_id': 'object'}

In [7]:
dept = '38'

td007 =pd.read_csv(data_dir/'depts'/dept/'td007_paroi_opaque.csv',dtype=str)
td008 =pd.read_csv(data_dir/'depts'/dept/'td008_baie.csv',dtype=str)

In [8]:
td008_types = {'id': 'str',
               'td007_paroi_opaque_id': 'str',
               'reference': 'str',
               'td008_baie_id': 'str',
               'deperdition': 'float',
               'tv009_coefficient_transmission_thermique_vitrage_id': 'category',
               'presence_survitrage': 'str',
               'coefficient_transmission_thermique_baie': 'float',
               'tv010_coefficient_transmission_thermique_baie_id': 'category',
               'tv011_resistance_additionnelle_id': 'category',
               'tv012_coef_transmission_thermique_baie_protection_solaire_id': 'category',
               'surface': 'float',
               'perimetre': 'float',
               'tv013_valeur_pont_thermique_id': 'category',
               'facteur_solaire': 'float',
               'tv021_facteur_solaire_id': 'category',
               'tv022_coefficient_masques_proches_id': 'category',
               'coefficient_masques_lointains_non_homogenes': 'category',
               'tv023_coefficient_masques_lointains_homogenes_id': 'category',
               'tv020_coefficient_orientation_id': 'category'}


def merge_td008_tr_tv(td008):
    from assets_orm import DPEMetaData
    meta = DPEMetaData()
    table = td008.copy()
    table = table.rename(columns={
        'tv012_coef_transmission_thermique_baie_protection_solaire_id': 'tv012_coefficient_transmission_thermique_baie_protection_solaire_id'})
    table = meta.merge_all_tr_tables(table)

    table = meta.merge_all_tv_tables(table)

    table = table.astype({k: v for k, v in td008_types.items() if k in table})
    table = table.loc[:, ~table.columns.duplicated()]

    return table

{'tv001_coefficient_reduction_deperditions':     tv001_code       tv001_aiu_aue tv001_aiu_aue_min tv001_aiu_aue_max  \
 0    TV001_001           Extérieur               NaN               NaN   
 1    TV001_002       Paroi enterré               NaN               NaN   
 2    TV001_003      Vide Sanitaire               NaN               NaN   
 3    TV001_004  Bâtiment adjacents               NaN               NaN   
 4    TV001_005              ? 0,25               NaN              0.25   
 ..         ...                 ...               ...               ...   
 257  TV001_258    10,00 <  ? 25,00             10.00             25.00   
 258  TV001_259       25,00 <  ? 50             25.00             50.00   
 259  TV001_260             50,00 <             50.00               NaN   
 260  TV001_261         Terre-plein               NaN               NaN   
 261  TV001_262             Aue = 0               NaN               NaN   
 
     tv001_uv_ue tv001_aue_isole tv001_aiu_isole tv001

In [ ]:
pd.concat(v)

In [111]:
meta.trtv_table_dict['tr014_type_parois_opaque']

,tr014_code,tr014_sous_type,tr014_type_parois_opaque_id
0,TR014_001,Paroi verticale extérieure,1
1,TR014_002,Paroi verticale intérieure,2
2,TR014_003,Plancher bas,3
3,TR014_004,Plancher haut,4
4,TR014_005,Plancher intermédiaire,5


In [109]:

list(meta.trtv_table_dict.keys())

['tv001_coefficient_reduction_deperditions',
 'tv002_local_non_chauffe',
 'tv002_local_non_chauffe_type_batiment',
 'tv003_umur',
 'tv004_umur0',
 'tv004_umur0_materiaux',
 'tv005_upb',
 'tv006_upb0',
 'tv007_uph',
 'tv007_uph_type_toit',
 'tv008_uph0',
 'tv009_coefficient_transmission_thermique_vitrage',
 'tv009_ug_orientation',
 'tv009_ug_remplissage',
 'tv009_ug_traitement_vitrage',
 'tv009_ug_type_vitrage',
 'tv010_coefficient_transmission_thermique_baie',
 'tv010_uw_type_baie',
 'tv010_uw_type_materiaux',
 'tv010_uw_type_menuiserie',
 'tv011_resistance_additionnelle',
 'tv012_coefficient_transmission_thermique_baie_protection_solaire',
 'tv013_valeur_pont_thermique',
 'tv013_valeur_pont_thermique_isolation_mur',
 'tv013_valeur_pont_thermique_isolation_plancher_bas',
 'tv013_valeur_pont_thermique_retour_isolation',
 'tv013_valeur_pont_thermique_type_liaison',
 'tv013_valeur_pont_thermique_type_pose',
 'tv014_permeabilite',
 'tv015_valeur_conventionnelle_renouvellement_air',
 'tv016

In [112]:
meta.trtv_table_dict

tv = pd.ExcelWriter('tv_tables.xlsx')
tr = pd.ExcelWriter('tr_tables.xlsx')

tr_tv_red =dict()
for k,v in meta.trtv_table_dict.items():
    red = k.split('_')[0]
    
    l=tr_tv_red.get(red,[])
    l.append(v)
    tr_tv_red[red]=l
    
for k,v in tr_tv_red.items():
    v.sort(key=lambda x:x.shape[1])
    v=v[-1]
    tr_tv_red[k]=v

    if k.startswith('tv'):
        v.to_excel(tv,sheet_name=k)
    elif k.startswith('tr'):
        v.to_excel(tr,sheet_name=k)

    

tv.save()
tr.save()

In [153]:

td011_types = {'td011_installation_chauffage_id': 'str',
               'td006_batiment_id': 'str',
               'tr003_type_installation_chauffage_id': 'category',
               'surface_chauffee': 'float',
               'nombre_appartements_echantillon': 'float',
               'surface_habitable_echantillon': 'float',
               'tv025_intermittence_id': 'category',
}

td012_types = {'id': 'str',
               'systeme_chauffage_cogeneration_id': 'string',
               'td011_installation_chauffage_id': 'str',
               'tr004_type_energie_id': 'category',
               'tv045_conversion_kwh_co2_id': 'category',
               'tv046_evaluation_contenu_co2_reseaux_id': 'category',
               'rendement_emission_systeme_chauffage': 'float',
               'tv028_rendement_emission_systeme_chauffage_id': 'category',
               'rendement_distribution_systeme_chauffage': 'str',
               'tv029_rendement_distribution_systeme_chauffage_id': 'category',
               'tv030_rendement_regulation_systeme_chauffage_id': 'category',
               'rendement_generation': 'float',
               'tv031_rendement_generation_id': 'category',
               'presence_regulation': 'int',
               'coefficient_performance': 'float',
               'tv032_coefficient_performance_id': 'category',
               'tv033_coefficient_correction_regulation_id': 'category',
               'tv034_temperature_fonctionnement_chaudiere_100_id': 'category',
               'tv035_temperature_fonctionnement_chaudiere_30_id': 'category',
               'rpn': 'float',
               'rpint': 'float',
               'qp0': 'float',
               'puissance_veilleuse': 'float',
               'tv036_puissance_veilleuse_id': 'category',
               'puissance_nominale': 'float',
               'tv038_puissance_nominale_id': 'category',
               'consommation_chauffage': 'float'}

# ===================== DICTIONARIES OF NORMALIZATION AND SIMPLIFICATION OF FIELDS ====================================

# dictionaries used to normalized chauffage names with simple labels.
gen_ch_lib_simp_dict = {'chaudiere gaz standard': 'chaudiere gaz standard',
                        'poele ou insert bois': 'poele ou insert bois',
                        'convecteurs electriques nfc': 'generateurs a effet joule',
                        'chaudiere fioul standard': 'chaudiere fioul standard',
                        'panneaux rayonnants electriques nfc': 'generateurs a effet joule',
                        'chaudiere gaz condensation': 'chaudiere gaz performante(condensation ou basse temperature)',
                        'chaudiere gpl condensation': 'chaudiere gpl performante(condensation ou basse temperature)',
                        'chaudiere gpl standard': 'chaudiere gpl standard',
                        'radiateurs electriques': 'generateurs a effet joule',
                        'reseau de chaleur': 'reseau de chaleur',
                        'chaudiere gaz basse temperature': 'chaudiere gaz performante(condensation ou basse temperature)',
                        'chaudiere gpl basse temperature': 'chaudiere gaz performante(condensation ou basse temperature)',
                        'autres emetteurs a effet joule': 'generateurs a effet joule',
                        'pac air/air': 'pac air/air',
                        'plafonds/planchers rayonnants electriques nfc': 'generateurs a effet joule',
                        'chaudiere bois': 'chaudiere bois',
                        'chaudiere fioul basse temperature': 'chaudiere fioul performante(condensation ou basse temperature)',
                        'chaudiere fioul condensation': 'chaudiere fioul performante(condensation ou basse temperature)',
                        'chaudiere electrique': 'chaudiere electrique',
                        'poele ou insert fioul/gpl': 'poele ou insert fioul/gpl',
                        'convecteurs bi-jonction': 'generateurs a effet joule', }

# GEN CH NORMALIZED DICT
gen_ch_normalized_lib_matching_dict = {"pac air/air": [('pac', 'PAC'), 'air/air', ('electricite', 'electrique')],
                                       "pac air/eau": [('pac', 'PAC'), 'air/eau', ('electricite', 'electrique')],
                                       "pac eau/eau": [('pac', 'PAC'), 'eau/eau', ('electricite', 'electrique')],
                                       "pac geothermique": [('pac', 'PAC'), (
                                           'geothermique', 'géothermique', 'géothermie', 'geothermie'),
                                                            ('electricite', 'electrique')],
                                       'panneaux rayonnants electriques nfc': ['panneau', ('electricite', 'electrique'),
                                                                               'nfc'],
                                       'radiateurs electriques': ['radiateur', ('electricite', 'electrique')],
                                       'plafonds/planchers rayonnants electriques nfc': [('plancher', 'plafond'),
                                                                                         ('electricite', 'electrique')],
                                       "convecteurs electriques nfc": ['convecteur', ('electricite', 'electrique'),
                                                                       'nfc'],
                                       "poele ou insert bois": [('poele', 'insert'), ('bois', 'biomasse')],
                                       "poele ou insert fioul/gpl": [('poele', 'insert'), ('fioul', 'gpl')],

                                       "autres emetteurs a effet joule": [('electricite', 'electrique')],
                                       "reseau de chaleur": ['reseau', 'chaleur'],
                                       "convecteurs bi-jonction": ['bi', 'jonction', ('electricite', 'electrique')],
                                       }

for type_chaudiere, type_chaudiere_keys in zip(['standard', 'basse temperature', 'condensation', 'non déterminee'],
                                               [('standard', 'classique'), 'basse temperature',
                                                ('condensation', 'condenseurs'), None]):
    for energie in ['fioul', 'gaz']:
        if type_chaudiere_keys is not None:
            gen_ch_normalized_lib_matching_dict[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie,
                                                                                            type_chaudiere_keys]
        else:
            gen_ch_normalized_lib_matching_dict[f'chaudiere {energie} {type_chaudiere}'] = ['chaudiere', energie
                                                                                            ]
    energie_gaz = gen_ch_normalized_lib_matching_dict[f'chaudiere gaz {type_chaudiere}'][1]
    energie_gaz = (energie_gaz, 'gpl', 'butane', 'propane')
    gen_ch_normalized_lib_matching_dict[f'chaudiere gaz {type_chaudiere}'][1] = energie_gaz

gen_ch_normalized_lib_matching_dict['chaudiere bois/biomasse'] = ['chaudiere',
                                                                  ('bois', 'biomasse')]
gen_ch_normalized_lib_matching_dict['chaudiere electrique'] = ['chaudiere',
                                                               ('electricite', 'electrique')]
pac_dict = {2.2: 'pac air/air',
            2.6: 'pac air/eau',
            3.2: "pac eau/eau",
            4.0: "pac géothermique"}

poele_dict = {0.78: 'poele ou insert bois',
              0.66: 'poele ou insert bois',
              0.72: "poele ou insert fioul/gpl"}


def merge_td011_tr_tv(td011):
    meta = DPEMetaData()
    table = td011.copy()
    table = meta.merge_all_tr_tables(table)
    table = meta.merge_all_tv_tables(table)
    table = table.rename(columns={'id': 'td011_installation_chauffage_id'})
    table = table.astype(td011_types)

    return table


def merge_td012_tr_tv(td012):
    meta = DPEMetaData()
    table = td012.copy()
    table = meta.merge_all_tr_tables(table)
    table = meta.merge_all_tv_tables(table)
    table = table.rename(columns={'id': 'td012_generateur_chauffage_id'})
    table = table.astype({k: v for k, v in td012_types.items() if k in table})

    return table


In [196]:
def merge_td001_dpe_id_system(td001, td006, td011, td012,td013,td014):
    td001 = td001.rename(columns={"id": "td001_dpe_id"})
    td006 = td006.rename(columns={"id": "td006_batiment_id"})
    td011 = td011.rename(columns={"id": "td011_installation_chauffage_id"})
    td012 = td012.rename(columns={"id": "td012_generateur_chauffage_id"})
    td013 = td013.rename(columns={"id": "td013_installation_ecs_id"})
    td014 = td014.rename(columns={"id": "td014_generateur_ecs_id"})

    td011 = td011.merge(td006[['td006_batiment_id', 'td001_dpe_id']], on='td006_batiment_id', how='left')
    td012 = td012.merge(td011[['td011_installation_chauffage_id', 'td001_dpe_id']], on='td011_installation_chauffage_id', how='left')

    td013 = td013.merge(td006[['td006_batiment_id', 'td001_dpe_id']], on='td006_batiment_id', how='left')
    td014 = td014.merge(td013[['td013_installation_ecs_id', 'td001_dpe_id']], on='td013_installation_ecs_id', how='left')


    return td001,td006,td011, td012,td013,td014

In [197]:
td001,td006,td011, td012,td013,td014 = merge_td001_dpe_id_system(td001, td006, td011, td012,td013,td014)

In [198]:
td013_types = {
               'td006_batiment_id': 'str',
               'tr005_type_installation_ecs_id': 'category',
               'nombre_appartements_echantillon': 'float',
               'surface_habitable_echantillon': 'float',
               'becs': 'float',
               'tv039_formule_becs_id': 'category',
               'surface_alimentee': 'float'}

td014_types = {'td014_generateur_ecs_id': 'str',
               'td013_installation_ecs_id': 'str',
               'tr004_type_energie_id': 'str',
               'tv045_conversion_kwh_co2_id': 'category',
               'tv046_evaluation_contenu_co2_reseaux_id': 'category',
               'coefficient_performance': 'float',
               'tv032_coefficient_performance_id': 'category',
               'rpn': 'float',
               'qp0': 'float',
               'puissance_veilleuse': 'float',
               'tv036_puissance_veilleuse_id': 'category',
               'tv037_puissance_necessaire_production_ecs_id': 'category',
               'puissance_nominale': 'float',
               'tv038_puissance_nominale_id': 'category',
               'tv040_rendement_distribution_ecs_id': 'category',
               'tv047_rendement_generation_ecs_id': 'category',
               'volume_stockage': 'float',
               'tv048_rendement_stockage_ecs_id': 'category',
               'tv049_perte_stockage_ecs_id': 'category',
               'tv041_coefficient_emplacement_fonctionnement_id': 'category',
               'tv043_pertes_stockage_id': 'category',
               'tv019_fecs_id': 'category',
               'tv027_pertes_recuperees_ecs_id': 'category',
}


def merge_td013_tr_tv(td013):
    meta = DPEMetaData()
    table = td013.copy()
    table = meta.merge_all_tr_tables(table)
    table = meta.merge_all_tv_tables(table)
    table = table.astype(td013_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table


def merge_td014_tr_tv(td014):
    meta = DPEMetaData()
    table = td014.copy()
    table = meta.merge_all_tr_tables(table)
    table = meta.merge_all_tv_tables(table)
    table = table.astype(td014_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table


In [199]:
td013 = merge_td013_tr_tv(td013)

td014 = merge_td014_tr_tv(td014)

In [206]:
table = td014.copy()

table = table.merge(td013[['tr005_description', 'td013_installation_ecs_id', 'surface_habitable_echantillon']],
                    on='td013_installation_ecs_id')

is_chaudiere = table.rpn > 0

gen_ecs_concat_txt_desc = table["tv027_type_installation"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc.loc[is_chaudiere] += 'chaudiere '

gen_ecs_concat_txt_desc += table['tv027_type_systeme'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv027_type_installation'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv032_type_generateur"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv036_type_generation'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv037_type_production'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv040_type_generateur'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv040_type_installation"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tr004_description"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv045_energie"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv047_type_generateur'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tr005_description'].astype('string').replace(np.nan, '') + ' '

gen_ecs_normalized_lib_matching_dict = {
    "ECS thermodynamique electrique(PAC ou ballon)": [
        ('pompe a chaleur', 'pac', 'thermodynamique', 'air extrait', 'air exterieur', 'air ambiant'),
        ('electricite', 'electrique')],
    "ballon a accumulation electrique": [('ballon', 'classique', 'accumulation'), ('electricite', 'electrique')],
    "ecs electrique indeterminee": [('electricite', 'electrique')],
    "ecs instantanee electrique": ['instantanee', ('electricite', 'electrique')],

    'chaudiere mixte gaz': ["chaudiere", 'mixte', "gaz"],
    'chaudiere mixte fioul': ["chaudiere", "mixte", "fioul"],
    'chaudiere mixte bois': [("bois", "biomasse")],

    'chauffe-eau gaz independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gaz"],
    'chauffe-eau gpl independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gpl"],

    'chauffe-eau fioul independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"),
                                      "fioul"],
    "ecs collective reseau chaleur": ["reseau", "chaleur"],

    'chaudiere gaz': ["chaudiere", "gaz"],
    'chaudiere gpl': ["chaudiere", "gpl"],

    'chaudiere fioul': ["chaudiere", "fioul"],

}

solaire_dict = dict()
for k, v in gen_ecs_normalized_lib_matching_dict.items():
    k_solaire = 'ecs solaire thermique et ' + k
    solaire_dict[k_solaire] = v + ['avec solaire']
gen_ecs_normalized_lib_matching_dict.update(solaire_dict)
gen_ecs_lib_simp_dict = {'ecs electrique indeterminee': 'ecs electrique indeterminee',
                         'chaudiere gaz': 'chaudiere gaz',
                         'ballon a accumulation electrique': 'ecs à effet joule electrique',
                         'chaudiere mixte gaz': 'chaudiere gaz',
                         'ECS thermodynamique electrique(PAC ou ballon)': 'ECS thermodynamique electrique(PAC ou ballon)',
                         'non affecte': 'non affecte',
                         'chaudiere fioul': 'chaudiere fioul',
                         'chaudiere mixte fioul': 'chaudiere fioul',
                         'chaudiere mixte bois': 'chaudiere mixte bois',
                         'ecs collective reseau chaleur': 'ecs collective reseau chaleur',
                         'ecs instantanee electrique': 'ecs à effet joule electrique',
                         'chauffe-eau gaz independant': 'chauffe-eau gaz independant',
                         'chauffe-eau fioul independant': 'chauffe-eau fioul independant',
                         }
solaire_dict = dict()
for k, v in gen_ecs_lib_simp_dict.items():
    k_solaire = 'ecs thermique solaire et ' + k
    v_solaire = 'ecs thermique solaire et ' + v
    solaire_dict[k_solaire] = v_solaire
gen_ecs_lib_simp_dict.update(solaire_dict)

sys_principal_scores = {'thermodynamique': 5,
                        'solaire': 4,
                        'chaudiere': 3,
                        'ballon a accumulation': 2,
                        'electrique indeterminee': 1,
                        'indépendant': 0,}

sys_principal_score_lib = dict()
for k in list(gen_ecs_normalized_lib_matching_dict.keys()):
    sys_principal_score_lib[k] = 0
    for term, score in sys_principal_scores.items():
        if term in k:
            sys_principal_score_lib[k] += score
sys_principal_score_lib['non affecte']=-1
gen_ecs_concat_txt_desc = gen_ecs_concat_txt_desc.str.lower().apply(lambda x: strip_accents(x))

table['gen_ecs_concat_txt_desc'] = gen_ecs_concat_txt_desc

table['gen_ecs_concat_txt_desc'] = table['gen_ecs_concat_txt_desc'].apply(lambda x: clean_str(x))

# calcul gen_ecs_lib_infer par matching score text.
unique_gen_ecs = table.gen_ecs_concat_txt_desc.unique()
gen_ecs_lib_infer_dict = {k: affect_lib_by_matching_score(k, gen_ecs_normalized_lib_matching_dict) for k in
                          unique_gen_ecs}
table['gen_ecs_lib_infer'] = table.gen_ecs_concat_txt_desc.replace(gen_ecs_lib_infer_dict)
is_pac = table.coefficient_performance > 2
table.loc[is_pac, 'gen_ecs_lib_infer'] = "ECS thermodynamique electrique(PAC ou ballon)"
ecs_ind = table.gen_ecs_lib_infer == 'ecs electrique indeterminee'
stockage = table.volume_stockage > 20
table.loc[ecs_ind & stockage, 'gen_ecs_lib_infer'] = 'ballon a accumulation electrique'
table.loc[ecs_ind & (~stockage), 'gen_ecs_lib_infer'] = 'ballon a accumulation electrique'
table['gen_ecs_lib_infer_simp'] = table.gen_ecs_lib_infer.replace(gen_ecs_lib_simp_dict)

table['type_energie_ecs'] = table['tr004_description']

table['score_gen_ecs_lib_infer'] = table['gen_ecs_lib_infer'].replace(sys_principal_score_lib).astype(float)

In [207]:
td014 =table

In [210]:
sys_ecs_principal_rename = {
    'td001_dpe_id': 'td001_dpe_id',
    'gen_ecs_lib_infer': 'sys_ecs_principal_gen_ecs_lib_infer',
    'gen_ecs_lib_infer_simp': 'sys_ecs_principal_gen_ecs_lib_infer_simp',
    'type_energie_ecs': 'sys_ecs_principal_type_energie_ecs',
    "tr005_description": 'sys_ecs_principal_type_installation_ecs',
    'nombre_generateurs': 'sys_ecs_principal_nb_generateur'
}

sys_ecs_secondaire_rename = {
    'td001_dpe_id': 'td001_dpe_id',
    'gen_ecs_lib_infer': 'sys_ecs_secondaire_gen_ecs_lib_infer',
    'gen_ecs_lib_infer_simp': 'sys_ecs_secondaire_gen_ecs_lib_infer_simp',
    'type_energie_ecs': 'sys_ecs_secondaire_type_energie_ecs',
    "tr005_description": 'sys_ecs_secondaire_type_installation_ecs',
    'nombre_generateurs': 'sys_ecs_secondaire_nb_generateur'
}

sys_ecs_tertiaire_concat_rename = {
    'td001_dpe_id': 'td001_dpe_id',
    'gen_ecs_lib_infer': 'sys_ecs_tertiaire_gen_ecs_lib_infer_concat',
    'gen_ecs_lib_infer_simp': 'sys_ecs_tertiaire_gen_ecs_lib_infer_simp_concat',
    'type_energie_ecs': 'sys_ecs_tertiaire_type_energie_ecs_concat',
    "tr005_description": 'sys_ecs_tertiaire_type_installation_ecs_concat',
    'nombre_generateurs': 'sys_ecs_tertiaire_nb_generateurs'
}

table = td014.copy()
is_solaire = table.gen_ecs_lib_infer.str.contains('ecs solaire thermique')
table['nombre_generateurs'] = 1
table.loc[is_solaire, 'nombre_generateurs'] = 2

cols = ['td001_dpe_id', 'gen_ecs_lib_infer_simp', 'gen_ecs_lib_infer', 'type_energie_ecs',
        'score_gen_ecs_lib_infer']
cols += ['tr005_description', 'surface_habitable_echantillon', 'nombre_generateurs', 'id_unique']

agg_cols = ['td001_dpe_id', 'gen_ecs_lib_infer', 'tr005_description']

table['id_unique'] = table.td001_dpe_id + table.gen_ecs_lib_infer.astype('string') + table.tr005_description.astype(
    'string')

is_unique = table.groupby('td001_dpe_id').td001_dpe_id.count() == 1
ist_unique = is_unique[is_unique].index

table_gen_unique = table.loc[table.td001_dpe_id.isin(ist_unique)][cols]

table_gen_multiple = table.loc[~table.td001_dpe_id.isin(ist_unique)][cols]

table_gen_multiple[agg_cols] = table_gen_multiple[agg_cols].astype(str)

agg = table_gen_multiple.groupby(agg_cols).agg({
    'gen_ecs_lib_infer_simp': 'first',
    'type_energie_ecs': 'first',
    'surface_habitable_echantillon': 'sum',
    "nombre_generateurs": 'sum',
    'score_gen_ecs_lib_infer': 'mean'

}).reset_index()

agg['id_unique'] = agg.td001_dpe_id + agg.gen_ecs_lib_infer + agg.tr005_description

sys_principal = agg.sort_values(['surface_habitable_echantillon', 'score_gen_ecs_lib_infer'],
                                ascending=False).drop_duplicates(subset='td001_dpe_id')

id_sys_principal = sys_principal.id_unique.unique().tolist()

sys_secondaires = agg.loc[~agg.id_unique.isin(id_sys_principal)]

sys_secondaire = sys_secondaires.sort_values(['surface_habitable_echantillon', 'score_gen_ecs_lib_infer'],
                                             ascending=False).drop_duplicates(
    'td001_dpe_id')

id_sys_secondaire = sys_secondaire.id_unique.unique().tolist()

sys_tertiaires = agg.loc[~agg.id_unique.isin(id_sys_principal + id_sys_secondaire)]

sys_tertiaire_concat = sys_tertiaires.groupby('td001_dpe_id').agg({
    'gen_ecs_lib_infer': lambda x: ' + '.join(list(set(x))),

    'gen_ecs_lib_infer_simp': lambda x: ' + '.join(list(set(x))),
    'type_energie_ecs': lambda x: ' + '.join(list(set(x))),
    'surface_habitable_echantillon': 'sum',
    "nombre_generateurs": 'sum',
    'tr005_description': lambda x: ' + '.join(list(set(x))),

}).reset_index()

sys_principal = sys_principal.append(table_gen_unique[sys_principal.columns])

sys_principal = sys_principal.rename(columns=sys_ecs_principal_rename)[sys_ecs_principal_rename.values()]

sys_secondaire = sys_secondaire.rename(columns=sys_ecs_secondaire_rename)[sys_ecs_secondaire_rename.values()]

sys_tertiaire_concat = sys_tertiaire_concat.rename(columns=sys_ecs_tertiaire_concat_rename)[
    sys_ecs_tertiaire_concat_rename.values()]

td001_sys_ecs = td001[['td001_dpe_id']].merge(sys_principal, on='td001_dpe_id', how='left')
td001_sys_ecs = td001_sys_ecs.merge(sys_secondaire, on='td001_dpe_id', how='left')
td001_sys_ecs = td001_sys_ecs.merge(sys_tertiaire_concat, on='td001_dpe_id', how='left')
nb_installation = td013.groupby('td001_dpe_id').td013_installation_ecs_id.count().to_frame(
    'nombre_installations')
td001_sys_ecs = td001_sys_ecs.merge(nb_installation, on='td001_dpe_id', how='left')

cols_end = sys_principal.columns.tolist() + sys_secondaire.columns.tolist() + sys_tertiaire_concat.columns.tolist()
cols_end = np.unique(cols_end).tolist()
cols_end.remove('td001_dpe_id')
td001_sys_ecs['nombre_generateur_total'] = td001_sys_ecs.sys_ecs_principal_nb_generateur
td001_sys_ecs['nombre_generateur_total'] += td001_sys_ecs.sys_ecs_secondaire_nb_generateur.fillna(0)
td001_sys_ecs['nombre_generateur_total'] += td001_sys_ecs.sys_ecs_tertiaire_nb_generateurs.fillna(0)

cols = ['sys_ecs_principal_type_energie_ecs',
        'sys_ecs_secondaire_type_energie_ecs',
        'sys_ecs_tertiaire_type_energie_ecs_concat']

td001_sys_ecs['mix_energetique_ecs'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                          is_unique=True, is_sorted=True)

cols = ['sys_ecs_principal_type_installation_ecs',
        'sys_ecs_secondaire_type_installation_ecs',
        'sys_ecs_tertiaire_type_installation_ecs_concat']

td001_sys_ecs['type_installation_ecs_concat'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                                   is_unique=True, is_sorted=True)

cols = ['sys_ecs_principal_gen_ecs_lib_infer',
        'sys_ecs_secondaire_gen_ecs_lib_infer',
        'sys_ecs_tertiaire_gen_ecs_lib_infer_concat']

td001_sys_ecs['gen_ecs_lib_infer_concat'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                               is_unique=True, is_sorted=True)

cols = ['sys_ecs_principal_gen_ecs_lib_infer_simp',
        'sys_ecs_secondaire_gen_ecs_lib_infer_simp',
        'sys_ecs_tertiaire_gen_ecs_lib_infer_simp_concat']

td001_sys_ecs['gen_ecs_lib_infer_simp_concat'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                                    is_unique=True, is_sorted=True)

isnull = td001_sys_ecs.sys_ecs_principal_nb_generateur.isnull()
is_multiple_install = td001_sys_ecs.nombre_installations > 1
td001_sys_ecs.loc[isnull, 'configuration_sys_ecs'] = pd.NA
td001_sys_ecs.loc[~isnull, 'configuration_sys_ecs'] = 'type de générateur unique/installation unique'
is_solaire = td001_sys_ecs.sys_ecs_principal_gen_ecs_lib_infer.str.contains('ecs solaire thermique')
isnull = td001_sys_ecs.sys_ecs_secondaire_nb_generateur.isnull()
td001_sys_ecs.loc[
    ~isnull | is_solaire, 'configuration_sys_ecs'] = 'types de générateur multiples/installation unique'
td001_sys_ecs.loc[(~isnull | is_solaire) & (
    is_multiple_install), 'configuration_sys_ecs'] = 'types de générateur multiples/installations multiples'

cols_first = [el for el in td001_sys_ecs.columns.tolist() if el not in cols_end]

cols = cols_first + cols_end

In [209]:
td001_sys_ecs[cols]

,nombre_installations,nombre_generateur_total,mix_energetique_ecs,type_installation_ecs_concat,gen_ecs_lib_infer_concat,gen_ecs_lib_infer_simp_concat,configuration_sys_ecs,sys_ecs_principal_gen_ecs_lib_infer,sys_ecs_principal_gen_ecs_lib_infer_simp,sys_ecs_principal_nb_generateur,...,sys_ecs_secondaire_gen_ecs_lib_infer_simp,sys_ecs_secondaire_nb_generateur,sys_ecs_secondaire_type_energie_ecs,sys_ecs_secondaire_type_installation_ecs,sys_ecs_tertiaire_gen_ecs_lib_infer_concat,sys_ecs_tertiaire_gen_ecs_lib_infer_simp_concat,sys_ecs_tertiaire_nb_generateurs,sys_ecs_tertiaire_type_energie_ecs_concat,sys_ecs_tertiaire_type_installation_ecs_concat,td001_dpe_id
0,1.0,1.0,Electricité,Un seul système d'ECS sans solaire,ballon a accumulation electrique,ecs à effet joule electrique,type de générateur unique/installation unique,ballon a accumulation electrique,ecs à effet joule electrique,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,291
1,2.0,4.0,Electricité,Deux systèmes d'ECS dans une maison ou un appa...,ballon a accumulation electrique,ecs à effet joule electrique,type de générateur unique/installation unique,ballon a accumulation electrique,ecs à effet joule electrique,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,423
2,1.0,1.0,Autres énergies,Un seul système d'ECS sans solaire,non affecte,non affecte,type de générateur unique/installation unique,non affecte,non affecte,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,435
3,1.0,1.0,Electricité,Un seul système d'ECS sans solaire,ballon a accumulation electrique,ecs à effet joule electrique,type de générateur unique/installation unique,ballon a accumulation electrique,ecs à effet joule electrique,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,493
4,1.0,1.0,Electricité,Un seul système d'ECS sans solaire,ballon a accumulation electrique,ecs à effet joule electrique,type de générateur unique/installation unique,ballon a accumulation electrique,ecs à effet joule electrique,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170231,1.0,1.0,Gaz,Un seul système d'ECS sans solaire,chaudiere gaz,chaudiere gaz,type de générateur unique/installation unique,chaudiere gaz,chaudiere gaz,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9431119
170232,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9431213
170233,1.0,1.0,Electricité,Un seul système d'ECS sans solaire,ballon a accumulation electrique,ecs à effet joule electrique,type de générateur unique/installation unique,ballon a accumulation electrique,ecs à effet joule electrique,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9431307
170234,1.0,1.0,Gaz,Un seul système d'ECS sans solaire,chaudiere gaz,chaudiere gaz,type de générateur unique/installation unique,chaudiere gaz,chaudiere gaz,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9431405


In [178]:
table.loc[,'gen_ecs_concat_txt_desc'].value_counts()

electricite electricite (hors electricite d'origine renouvelab un seul systeme d'ecs sans solaire                                                                                                   38633
electricite electricite (hors electricite d'origine renouvelab deux systemes d'ecs dans une maison ou un appartement                                                                                 4100
collective tous les systemes collective reseau collectif isole collective electricite electricite (hors electricite d'origine renouvelab electrique un seul systeme d'ecs sans solaire                176
electricite electricite d'origine renouvelable utilisee dans l un seul systeme d'ecs sans solaire                                                                                                      24
collective tous les systemes collective reseau collectif non isole collective electricite electricite (hors electricite d'origine renouvelab electrique un seul systeme d'ecs sans solaire      

In [173]:
table.count().to_dict()

{'td014_generateur_ecs_id': 109506,
 'td013_installation_ecs_id': 109506,
 'tr004_type_energie_id': 109506,
 'tv045_conversion_kwh_co2_id': 109506,
 'tv046_evaluation_contenu_co2_reseaux_id': 1195,
 'coefficient_performance': 107352,
 'tv032_coefficient_performance_id': 4513,
 'rpn': 107352,
 'qp0': 107352,
 'puissance_veilleuse': 107352,
 'tv036_puissance_veilleuse_id': 8784,
 'tv037_puissance_necessaire_production_ecs_id': 11952,
 'puissance_nominale': 109506,
 'tv038_puissance_nominale_id': 5019,
 'tv040_rendement_distribution_ecs_id': 32717,
 'tv047_rendement_generation_ecs_id': 30012,
 'volume_stockage': 109302,
 'tv048_rendement_stockage_ecs_id': 32717,
 'tv049_perte_stockage_ecs_id': 31582,
 'tv041_coefficient_emplacement_fonctionnement_id': 4475,
 'tv043_pertes_stockage_id': 20632,
 'tv019_fecs_id': 173,
 'tv027_pertes_recuperees_ecs_id': 23293,
 'td001_dpe_id': 109506,
 'tr004_code': 109506,
 'tr004_description': 109506,
 'tv045_code': 109506,
 'tv045_energie': 109506,
 'tv045

In [31]:
meta.trtv_table_dict['tv022_coefficient_masques_proches']

,tv022_code,tv022_rapport_l1_l2,tv022_largeur_baie_superieure,tv022_beta_gama,tv022_angle_superieur_30,tv022_fe1,tv022_type_masque,tv022_avance,tv0xx_orientation,tv022_coefficient_masques_proches_id
0,TV022_001,NaN,NaN,NaN,NaN,1.00,Absence de masque proche,NaN,NaN,1
1,TV022_002,NaN,NaN,NaN,NaN,0.40,Fond de balcon ou fond et flanc de loggias,< 1 m,Nord,2
2,TV022_003,NaN,NaN,NaN,NaN,0.50,Fond de balcon ou fond et flanc de loggias,< 1 m,Sud,3
3,TV022_004,NaN,NaN,NaN,NaN,0.45,Fond de balcon ou fond et flanc de loggias,< 1 m,Est ou Ouest,4
4,TV022_005,NaN,NaN,NaN,NaN,0.30,Fond de balcon ou fond et flanc de loggias,1 <= … < 2,Nord,5
5,TV022_006,NaN,NaN,NaN,NaN,0.40,Fond de balcon ou fond et flanc de loggias,1 <= … < 2,Sud,6
6,TV022_007,NaN,NaN,NaN,NaN,0.35,Fond de balcon ou fond et flanc de loggias,1 <= … < 2,Est ou Ouest,7
7,TV022_008,NaN,NaN,NaN,NaN,0.20,Fond de balcon ou fond et flanc de loggias,2 <= … < 3,Nord,8
8,TV022_009,NaN,NaN,NaN,NaN,0.30,Fond de balcon ou fond et flanc de loggias,2 <= … < 3,Sud,9
9,TV022_010,NaN,NaN,NaN,NaN,0.25,Fond de balcon ou fond et flanc de loggias,2 <= … < 3,Est ou Ouest,10


In [22]:
list(meta.trtv_table_dict.keys())

['tv001_coefficient_reduction_deperditions',
 'tv002_local_non_chauffe',
 'tv002_local_non_chauffe_type_batiment',
 'tv003_umur',
 'tv004_umur0',
 'tv004_umur0_materiaux',
 'tv005_upb',
 'tv006_upb0',
 'tv007_uph',
 'tv007_uph_type_toit',
 'tv008_uph0',
 'tv009_coefficient_transmission_thermique_vitrage',
 'tv009_ug_orientation',
 'tv009_ug_remplissage',
 'tv009_ug_traitement_vitrage',
 'tv009_ug_type_vitrage',
 'tv010_coefficient_transmission_thermique_baie',
 'tv010_uw_type_baie',
 'tv010_uw_type_materiaux',
 'tv010_uw_type_menuiserie',
 'tv011_resistance_additionnelle',
 'tv012_coefficient_transmission_thermique_baie_protection_solaire',
 'tv013_valeur_pont_thermique',
 'tv013_valeur_pont_thermique_isolation_mur',
 'tv013_valeur_pont_thermique_isolation_plancher_bas',
 'tv013_valeur_pont_thermique_retour_isolation',
 'tv013_valeur_pont_thermique_type_liaison',
 'tv013_valeur_pont_thermique_type_pose',
 'tv014_permeabilite',
 'tv015_valeur_conventionnelle_renouvellement_air',
 'tv016

In [20]:
td012.columns

Index(['id', 'systeme_chauffage_cogeneration_id',
       'td011_installation_chauffage_id', 'tr004_type_energie_id',
       'tv045_conversion_kwh_co2_id',
       'tv046_evaluation_contenu_co2_reseaux_id',
       'rendement_emission_systeme_chauffage',
       'tv028_rendement_emission_systeme_chauffage_id',
       'rendement_distribution_systeme_chauffage',
       'tv029_rendement_distribution_systeme_chauffage_id',
       'tv030_rendement_regulation_systeme_chauffage_id',
       'rendement_generation', 'tv031_rendement_generation_id',
       'presence_regulation', 'coefficient_performance',
       'tv032_coefficient_performance_id',
       'tv033_coefficient_correction_regulation_id',
       'tv034_temperature_fonctionnement_chaudiere_100_id',
       'tv035_temperature_fonctionnement_chaudiere_30_id', 'rpn', 'rpint',
       'qp0', 'puissance_veilleuse', 'tv036_puissance_veilleuse_id',
       'puissance_nominale', 'tv038_puissance_nominale_id',
       'consommation_chauffage'],
      dt

In [13]:
td007 = meta.merge_all_tv_tables(td007)
td007 = meta.merge_all_tr_tables(td007)

KeyError: 'tv001_code'

In [12]:
meta.trtv_table_dict['tv003_umur'].count()

tv003_code                      64
tv003_mur_isole                 22
tv003_annee_construction        45
tv003_annee_construction_min    45
tv003_annee_construction_max    45
tv003_annee_isolation           18
tv003_annee_isolation_min       18
tv003_annee_isolation_max       18
tv003_effet_joule               42
tv003_umur                      64
tv017_code                       0
tv017_t_ext_moyen                0
tv017_peta_cw                    0
tv017_dh14                       0
tv017_prs1                       0
tv003_umur_id                   64
dtype: int64

In [5]:
td007.tv017_code.value_counts()

H1    130370
H3         0
H2         0
Name: tv017_code, dtype: int64

In [16]:
td007['tr014_sous_type'].value_counts()

Paroi verticale extérieure    245457
Plancher bas                   82832
Plancher haut                  82716
Paroi verticale intérieure     21495
Plancher intermédiaire          7438
Name: tr014_sous_type, dtype: int64

In [14]:
td007.count().sort_values(ascending=False).to_dict()

{'tr014_sous_type': 439938,
 'tr014_code': 439938,
 'td006_batiment_id': 439938,
 'tr014_type_parois_opaque_id': 439938,
 'deperdition_thermique': 439938,
 'coefficient_transmission_thermique_paroi': 439938,
 'surface_paroi': 439938,
 'id': 439938,
 'reference': 439926,
 'epaisseur_isolation': 435416,
 'resistance_thermique_isolation': 429240,
 'tv001_code': 361729,
 'tv001_aiu_aue': 361729,
 'tv001_valeur': 361729,
 'tv001_coefficient_reduction_deperditions_id': 361729,
 'coefficient_transmission_thermique_paroi_non_isolee': 335701,
 'tv004_umur0_id': 243518,
 'tv004_epaisseur': 243518,
 'tv004_code': 243518,
 'tv004_materiaux': 243518,
 'tv004_umur': 243518,
 'tv003_umur_id': 191403,
 'tv003_code': 191403,
 'tv003_umur': 191403,
 'tv003_annee_construction_max': 134474,
 'tv003_annee_construction_min': 134474,
 'tv003_annee_construction': 134474,
 'tv017_code': 130370,
 'tv017_peta_cw': 130370,
 'tv017_dh14': 130370,
 'tv017_prs1': 130370,
 'tv017_t_ext_moyen': 130370,
 'tv002_type_ba

In [23]:
tv004=meta.trtv_table_dict['tv004_umur0']

In [11]:
td007.tr014_type_parois_opaque_id.iloc[0]

'1'

In [12]:
td007.columns.sort_values()

Index(['coefficient_transmission_thermique_paroi',
       'coefficient_transmission_thermique_paroi_non_isolee',
       'deperdition_thermique', 'epaisseur_isolation', 'id', 'reference',
       'resistance_thermique_isolation', 'surface_paroi', 'td006_batiment_id',
       'tr014_code', 'tr014_sous_type', 'tr014_type_parois_opaque_id',
       'tv001_aiu_aue', 'tv001_aiu_aue_max', 'tv001_aiu_aue_min',
       'tv001_aiu_isole', 'tv001_aue_isole', 'tv001_code',
       'tv001_coefficient_reduction_deperditions_id', 'tv001_uv_ue',
       'tv001_valeur', 'tv002_code', 'tv002_local_non_chauffe',
       'tv002_local_non_chauffe_id', 'tv002_type_batiment', 'tv002_uvue',
       'tv003_annee_construction', 'tv003_annee_construction_max',
       'tv003_annee_construction_min', 'tv003_annee_isolation',
       'tv003_annee_isolation_max', 'tv003_annee_isolation_min', 'tv003_code',
       'tv003_effet_joule', 'tv003_mur_isole', 'tv003_umur', 'tv003_umur_id',
       'tv004_code', 'tv004_epaisseur', 'tv

In [31]:
tv004.tv004_umur0_id.iloc[0]

1

In [29]:
td007.merge(tv004,on = 'tv004_umur0_id')

,id,td006_batiment_id,tr014_type_parois_opaque_id,reference,deperdition_thermique,tv001_coefficient_reduction_deperditions_id,tv002_local_non_chauffe_id,coefficient_transmission_thermique_paroi,coefficient_transmission_thermique_paroi_non_isolee,tv003_umur_id,...,tv008_uph0,tv008_est_efface,tr014_code,tr014_sous_type,tr014_est_efface,tv004_code_y,tv004_epaisseur_y,tv004_umur_y,tv004_est_efface_y,tv004_materiaux_y


In [5]:
td007.columns

NameError: name 'table' is not defined

In [10]:
foreign_ids

[]

In [10]:
td007.tr014_type_parois_opaque_id.iloc[0]

'1'

In [16]:
td007.tv006_code.value_counts().sort_values()

TV006_012    0
TV006_011    0
TV006_010    0
TV006_009    0
TV006_008    0
TV006_007    0
TV006_006    0
TV006_005    0
TV006_004    0
TV006_003    0
TV006_002    0
TV006_001    0
Name: tv006_code, dtype: int64

In [8]:
td007.tv007_annee_construction.value_counts()

à partir de 2006    0
à partir de 2000    0
jusqu'à 1988        0
de 1989 à 2000      0
2000 à 2005         0
1988 à 2000         0
1982 à 1988         0
1977 à 1982         0
1974 à 1977         0
1948 à 1974         0
Name: tv007_annee_construction, dtype: int64

In [ ]:
meta.trtv_table_di